In [1]:
import csv
import glob
import os
import os.path
from subprocess import call
from os import walk
import json
import random

In [2]:
# breed list 

workDIr = os.path.abspath('.')
#==============
breed_list = []
#==============
breed_list_index = {}
#==============
class_list = {}
#==============
index_count = 0



for path, dir, files in os.walk(workDIr):
    for filename in files:
        ext = os.path.splitext(filename)[-1]

        if ext == '.json':
            #print("%s/%s" % (path, filename))
            json_path = path + '/' + filename
            #print(json_path)

            try:
                with open(json_path) as json_file:
                    json_data = json.load(json_file)
                    breed_name = json_data['metadata']['animal']['breed']

                    # breed list with index
                    if breed_name not in breed_list_index:
                        breed_list_index[breed_name] = index_count
                        breed_list.append(breed_name)
                        index_count += 1
                        
                    # breed class
                    if breed_name not in class_list.keys():
                        class_list[breed_name] = []
                    text_form = filename.split('-')[1]  + '/' + filename.split('.')[0]
                    #print(filename.split('.')[0])
                    class_list[breed_name].append(text_form)
                    
            except:
                print(filename)
                #pass

#print(breed_list_index)
print(class_list['골든 리트리버'])
#print(breed_list)


cat-lying-001754.mp4.json
dog-turn-000244.mp4.json
['sit/dog-sit-000890', 'heading/dog-heading-000179', 'bodylower/dog-bodylower-003771', 'walkrun/dog-walkrun-003019']


In [3]:
# 다 더해서 2500개 나오면 됨
def sum_calc(a):
    sum = 0
    for i in a:
        sum += len(a[i])

    return sum

print(sum_calc(class_list))

2518


In [6]:
subs_list = {}
#len(glob.glob(item + "/*jpg")
def substract_list(a):
    for key in a:
        for item in a[key]:
            if len(glob.glob(item + "/*jpg")) < 10:
                #print(item)
                print('{} : {}, '.format(key, item))
                #pass
            elif len(a[key]) < 4:
                #여러번 프린트하는건 무시..
                print('{} : {}, '.format(key, a[key]))
            else:
                if key not in subs_list.keys():
                    subs_list[key] = []
                #print(filename.split('.')[0])
                subs_list[key].append(item)

                #subs_list[key] = item


substract_list(class_list)
sum_calc(subs_list)

#print(subs_list)

푸들 : tailing/dog-tailing-000556, 
푸들 : tailing/dog-tailing-002473, 
푸들 : tailing/dog-tailing-001415, 
코리안 숏헤어 : laydown/cat-laydown-002822, 
코리안 숏헤어 : grooming/cat-grooming-001893, 
웰시 코기 (펨브록) : bodylower/dog-bodylower-000656, 
웰시 코기 (펨브록) : lying/dog-lying-001308, 
닥스훈트 : tailing/dog-tailing-002733, 
닥스훈트 : tailing/dog-tailing-002863, 
닥스훈트 : bodylower/dog-bodylower-002475, 
말티즈 : mounting/dog-mounting-002732, 
말티즈 : feetup/dog-feetup-007634, 
말티즈 : turn/dog-turn-000089, 
믹스견 : tailing/dog-tailing-002613, 
믹스견 : sit/dog-sit-002323, 
포메라니안 : bodylower/dog-bodylower-002259, 
포메라니안 : lying/dog-lying-001393, 
파피용 : feetup/dog-feetup-001312, 
프렌치 불독 : lying/dog-lying-002095, 
노르웨이숲 : grooming/cat-grooming-000098, 
비숑 프리제 : feetup/dog-feetup-008044, 
비숑 프리제 : turn/dog-turn-002933, 
비숑 프리제 : sit/dog-sit-002585, 
비숑 프리제 : sit/dog-sit-002611, 
비숑 프리제 : footup/dog-footup-002409, 
비숑 프리제 : sit/dog-sit-002772, 
비숑 프리제 : sit/dog-sit-002673, 
러시안블루 : grooming/cat-grooming-002912, 
러시안블루 : grooming

2462

In [7]:
split_rate = 0.8
trainRaw = {}
valRaw = {}          
arr = []

# class_list = {'행복/즐거움': [], '편안/안정': [], '화남/불쾌': [], '불안/슬픔': [], '공격성': [], '공포': []}

def train_val_shuffle(a):
    
    for clss in a:           
        cls_len = len(a[clss])
        split_point = int(split_rate * cls_len)  

        #list = a[clss]

        list = [item + " {} {}".format(len(glob.glob(item + "/*jpg")), breed_list_index[clss]) for item in a[clss]]



        random.shuffle(list)                # 섞어주기
        trainRaw[clss] = list[:split_point] # 80%
        valRaw[clss] = list[split_point:]   # 20%
        # couting each class_list
        arr.append([clss, len(a[clss])])
        #print('{} : {}, train: {}, val: {}'.format(clss, len(a[clss]), len(trainRaw[clss]), len(valRaw[clss])))    



# substitute class_list to subs_breed
train_val_shuffle(subs_list)

In [8]:
with open('breed_train_pkn1.txt', 'w') as f:
    for clss in trainRaw:
        for item in trainRaw[clss]:
            f.write("%s\n" % item)

In [9]:
with open('breed_val_pkn1.txt', 'w') as f:
    for clss in valRaw:
        for item in valRaw[clss]:
            f.write("%s\n" % item)

In [10]:
# Object detection Train set
CNT = 10
with open('train10.txt', 'w') as f:
    for clss in trainRaw:
        for item in trainRaw[clss]:
            file_path, frames, clss = item.split(' ')            
            # os.listdir 지정한 디렉토리 내의 모든 파일과 디렉토리 리스트 리턴
            file_list = os.listdir(file_path)

            if len(file_list) == 0:
                print(file_path)
            else:
                # choose random 10
                sampling = random.choices(file_list, k=CNT)
                #print(sampling)
    

            for itemf in sampling:
                f.write("{}\n".format(os.path.join(file_path,itemf, clss)))

In [11]:
# Object detection Val set
with open('val10.txt', 'w') as f:
    for clss in valRaw:
        for item in valRaw[clss]:
            file_path, frames, clss = item.split(' ')            
            file_list = os.listdir(file_path)

            if len(file_list) == 0:
                print(file_path)
            else:
                sampling = random.choices(file_list, k=CNT)
            # print(clss)
            #if clss == 'cat':
            #    clss = '0'                
            #else:
            #    clss = '1'                

            for itemf in sampling:
                f.write("{}\n".format(os.path.join(file_path,itemf, clss)))

In [12]:
with open('breed_list_index.txt', 'w') as f:
    for clss in breed_list_index:
            f.write("%s\n" % clss)